## Construction of Homological Scaffold and calculate NodalPSS

In this notebook we calculate Nodal PSS <br>
Code courtesy of Homological Scaffold construction: __[G.Petri](https://github.com/lordgrilo/Holes)__.

In [1]:
import sys, os, re, time
import pandas as pd
import math
import numpy as np
from random import random
import networkx as nx
import pickle as pk
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.multitest as multitest

sys.path.append('../')
import Holes as ho

In [2]:
def HomoScaffold(G, gen, outfilename):
    ScafH = nx.Graph()
    # ScafH.add_nodes_from(G)
    edges = []
    for c in gen[1]:
        edges.extend(c.cycles())

    for e in edges:
        u , v = int(e[0]) , int(e[1])
        if ScafH.has_edge(u,v):
            ScafH[u][v]['weight'] += 1
        else:
            ScafH.add_edge(u, v, weight=1)
            ScafH[u][v]['persistence'] = 0

    for cy in gen[1]:
        cyc =  cy.cycles()
        unique_integers = [[int(x) for x in sublist] for sublist in cyc]
        for sublist in cyc:
            u,v = int(sublist[0]) , int(sublist[1])
            if ScafH.has_edge(u,v):
                ScafH[u][v]['persistence'] += cy.persistence_interval()

    with open(outfilename,'wb') as f:
        pk.dump(ScafH, f)
    return "Stored Homological Scaffold"

In [3]:
dataset = 'MPI_LEMON'
# dataset = 'ABIDE'

path_file = f'../Data/{dataset}/FCM_DistMat/'
files_list = os.listdir(path_file)
print(len(files_list)) 

RSNs_details = pd.read_csv('../Data/SchaeferAtlas_Regions_details.csv')
RSNs7 = RSNs_details['RSN'].unique().tolist()
result_dict = {key: RSNs_details.loc[RSNs_details['RSN'] == key, 'Node_number'].tolist() for key in RSNs_details['RSN'].unique()}
RSN_node_details = { rsn : {i:result_dict[rsn][i] for i in range(len(result_dict[rsn]))} for rsn in RSNs7}
print(len(result_dict),[ len(x) for x in result_dict.values()],RSNs7)

225
7 [29, 35, 26, 22, 12, 30, 46] ['Visual', 'Somato Motor', 'Dorsal Attention', 'Salient Ventral Attention', 'Limbic', 'Control', 'Default']


### Holes
We first compute the scaffold. <br>
Aggregate the homological scaffold and write it to file.

In [5]:
t0 = time.time()
path = f'../OutputFiles/PosCorr/{dataset}/Output_RSNs/HomoScaffold/'
for RSN in RSNs7:
    t1 = time.time()
    indices = result_dict[RSN]
    print(RSN, len(indices))
    rsn = RSN.replace(' ','')
    for i in range(len(files_list)):
        t2 = time.time()
        SubID = files_list[i].split('.')[0].split('_')[-1]
        DisMat = pd.read_csv(path_file + files_list[i], header = None, sep = ',').values
        DisMat_rsn = DisMat[np.ix_(indices, indices)]
        DisMat_rsn[DisMat_rsn > np.sqrt(2)] = 0
        G = nx.from_numpy_array(DisMat_rsn)
        num_components = nx.number_connected_components(G)
        print(DisMat_rsn.shape,'num_components=', num_components,'\t', G)
        fil = ho.filtrations.upward_limited_weight_clique_rank_filtration(G,3)
        outpath = path + f'{rsn}/'
        os.makedirs(outpath + 'Filtration/', exist_ok= True)
        clique_dictionary_file = outpath + f'Filtration/Sub_{SubID}_filtration.pck'
        with open(clique_dictionary_file,'wb') as f:
            pk.dump(fil,f, protocol=2)
        print('Filtration done ', time.time() - t2)
        hom_dim = 1
        dataset_tag = f'Sub_{SubID}'
        output_dir = outpath
        
        # Compute the generators
        ho.persistent_homology_calculation(clique_dictionary_file, hom_dim, dataset_tag, output_dir, m1=512, m2=2048, save_generators=True)
        print('persistent_homology_calculation done ', time.time() - t2)
        # Dump the results in this file
        gen_file = outpath + f'gen/generators_Sub_{SubID}_.pck'
        with open(gen_file, 'rb') as f:
            gen = pk.load(f)
        os.makedirs(outpath + 'Scaffolds/', exist_ok= True)
        OutHomScaPath = outpath + f'Scaffolds/JPScaffold_Sub_{SubID}.pck'
        homscaf = HomoScaffold(G, gen, OutHomScaPath)
        print(f'Done for {SubID}', time.time() - t2, '-'*60 )
        break
    # print(f'Done for {rsn}', time.time() - t1 )
    break
print('Done for ', dataset, time.time() - t0) 

Visual 29
(29, 29) num_components= 1 	 Graph with 29 nodes and 380 edges
Preliminary scan of edge weights to define filtration steps...
Constructing filtration...
Max filtration value: 378
Clique dictionary created.
Filtration done  1.2714788913726807
0
persistent_homology_calculation done  8.5032377243042
Done for 32301 8.508232116699219 ------------------------------------------------------------
Done for  MPI_LEMON 8.508232116699219


Child returned 0


### Compute Nodal PSS

In [3]:
t0 = time.time()
outpath = f'../OutputFiles/PosCorr/{dataset}/Output_RSNs/HomoScaffold/'

for RSN in RSNs7:
    t1 = time.time()
    indices = result_dict[RSN]
    NNodes = len(indices)
    print(RSN, NNodes)
    rsn = RSN.replace(' ','')
    NPSSoutdict = {'Nodes': [n for n in range(NNodes)]}
    for i in range(len(files_list)):
        t2 = time.time()
        SubID = files_list[i].split('.')[0].split('_')[-1]
        inpath = f'../OutputFiles/PosCorr/{dataset}/Output_RSNs/HomoScaffold/{rsn}/'
        OutHomScaPath = inpath + f'Scaffolds/JPScaffold_Sub_{SubID}.pck'
        with open(OutHomScaPath, 'rb') as f:
            ScafH = pk.load(f)
        HSnode = dict(ScafH.degree(weight='persistence'))
        NodalPSS = {n: 0 for n in range(NNodes)}
        for n in range(NNodes):
            if n in HSnode.keys():
                NodalPSS[n] = HSnode[n]/2
        NPSSoutdict[SubID] = NodalPSS.values()

    NPSSfilename = f'{rsn}_NodalPSS_HomoScaffold.csv'
    NPSStOutdf = pd.DataFrame(NPSSoutdict)
    NPSStOutdf.to_csv(outpath + NPSSfilename, index = None)
    # break

Visual 29
Somato Motor 35
Dorsal Attention 26
Salient Ventral Attention 22
Limbic 12
Control 30
Default 46


## Statistical test

In [4]:
#------------------------------------------------
# Function to find mean an standard error 
#------------------------------------------------

def Node_Groupdiff(group1,group2,Infile,g1name,g2name,NNodes):
    p_val = []
    Gr1,Gr2 = [[],[]],[[],[]]
    for i in range(NNodes):
        data1,data2 = [],[]
        for sub1 in group1:
            data1.append(Infile[sub1][i])
        Gr1[0].append(np.mean(data1))
        Gr1[1].append(stats.sem(data1))
        
        for sub2 in group2:
            data2.append(Infile[sub2][i])
        Gr2[0].append(np.mean(data2))
        Gr2[1].append(stats.sem(data2))
        p_val.append(stats.ttest_ind(data1,data2,equal_var=False)[1])
        
    correction = multitest.multipletests(p_val, alpha=0.05, method = 'fdr_bh')
    fdr = correction[0]
    p_v_corrected = correction[1]

    nodes = [i for i in range(NNodes)]
    Gdiff = {'Nodes': nodes, 'p_values':p_val, 'fdr_corrected_p_val':p_v_corrected, f'{g1name}_mean': Gr1[0], f'{g1name}_sd':Gr1[1], f'{g2name}_mean':Gr2[0], f'{g2name}_sd':Gr2[1]}
    return pd.DataFrame(Gdiff)

In [5]:
if dataset == 'MPI_LEMON':
    Detailsfile = pd.read_csv('../Data/MPI_LEMON/MPILemon_Subject_details.csv')
    Young = list(map(str, list(Detailsfile.loc[Detailsfile['Cohort'] == 'Young','Subject'])))
    Elder = list(map(str, list(Detailsfile.loc[Detailsfile['Cohort'] == 'Elderly','Subject'])))
    All_subs = Young + Elder
    Group1, Group2 = 'Young', 'Elderly'
    GR1, GR2 = Young, Elder

elif dataset == 'ABIDE':
    Detailsfile = pd.read_csv('../Data/ABIDE/ABIDE_Subject_details.csv')
    ASD = list(map(str, list(Detailsfile.loc[Detailsfile['Cohort'] == 'ASD','Subject identifier'])))
    Healthy = list(map(str, list(Detailsfile.loc[Detailsfile['Cohort'] == 'HC','Subject identifier'])))
    All_subs = ASD + Healthy
    Group1, Group2 = 'ASD', 'Healthy'
    GR1, GR2 = ASD, Healthy

print(Group1, len(GR1), '\t', Group2, len(GR2), '\tTotal:', len(GR1+GR2))
print(GR1[0], GR2[0], outpath)

Young 153 	 Elderly 72 	Total: 225
32302 32301 ../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/


In [6]:
t0 = time.time()

for RSN in RSNs7:
    t1 = time.time()
    indices = result_dict[RSN]
    NNodes = len(indices)
    rsn = RSN.replace(' ','')
    NPSSfilename = f'{rsn}_NodalPSS_HomoScaffold.csv'
    Infile = pd.read_csv(outpath + NPSSfilename)
    print(NNodes, Infile.shape)
    
    df_NPSS = Node_Groupdiff(GR1, GR2, Infile, Group1, Group2,NNodes)
    df_NPSS = pd.DataFrame(df_NPSS)
    df_NPSS.to_csv(outpath + f'p_values_{rsn}_NodalPSS_HomoScaffold.txt',sep = '\t', index=None)
    
    print(outpath +f'p_values_{rsn}_NodalPSS_HomoScaffold.txt', f'\n Done for {rsn}  {df_NPSS.shape}\t Time:', time.time() - t0)
    # break
print('Done')

29 (29, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_Visual_NodalPSS_HomoScaffold.txt 
 Done for Visual  (29, 7)	 Time: 0.09370732307434082
35 (35, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_SomatoMotor_NodalPSS_HomoScaffold.txt 
 Done for SomatoMotor  (35, 7)	 Time: 0.18959379196166992
26 (26, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_DorsalAttention_NodalPSS_HomoScaffold.txt 
 Done for DorsalAttention  (26, 7)	 Time: 0.2813551425933838
22 (22, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_SalientVentralAttention_NodalPSS_HomoScaffold.txt 
 Done for SalientVentralAttention  (22, 7)	 Time: 0.33707427978515625
12 (12, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_Limbic_NodalPSS_HomoScaffold.txt 
 Done for Limbic  (12, 7)	 Time: 0.37084388732910156
30 (30, 226)
../OutputFiles/PosCorr/MPI_LEMON/Output_RSNs/HomoScaffold/p_values_Control_NodalPSS_HomoSca

In [7]:
NIBS_file = pd.read_csv('../Data/NIBS_Identified_ROIs.csv')
nibs = NIBS_file.loc[NIBS_file[dataset] == True, 'Node_ID'].tolist()
print(dataset, len(nibs), nibs)

print(Group1, len(GR1), '\t', Group2, len(GR2), '\tTotal:', len(GR1+GR2),'\n')

FDRcount, CoinNIBScount = 0,0
for RSN in RSNs7:
    rsn = RSN.replace(' ','')
    indices = result_dict[RSN]
    NNodes = len(indices)
    df = pd.read_csv(outpath + f'p_values_{rsn}_NodalPSS_HomoScaffold.txt', sep = '\t')
    filtered_values = df.loc[df['fdr_corrected_p_val'] < 0.05, 'Nodes']
    node_ids_fdr = [RSN_node_details[RSN][i] for i in filtered_values]
    nodesno = set(nibs).intersection(set(node_ids_fdr))
    FDRcount += len(filtered_values)
    CoinNIBScount += len(nodesno)
    print("FDR-corrected:", len(filtered_values), '\t Coincide with NIBS:', len(nodesno),'\t', rsn)

print('\nOverall:\t', "FDR-corrected:", FDRcount, '\t Coincide with NIBS:', CoinNIBScount)

MPI_LEMON 42 [19, 23, 28, 32, 34, 35, 36, 37, 38, 39, 45, 50, 60, 61, 70, 81, 90, 92, 124, 125, 126, 128, 130, 131, 132, 133, 139, 140, 142, 143, 144, 150, 157, 164, 165, 166, 173, 175, 181, 183, 194, 195]
Young 153 	 Elderly 72 	Total: 225 

FDR-corrected: 2 	 Coincide with NIBS: 0 	 Visual
FDR-corrected: 26 	 Coincide with NIBS: 11 	 SomatoMotor
FDR-corrected: 5 	 Coincide with NIBS: 1 	 DorsalAttention
FDR-corrected: 7 	 Coincide with NIBS: 2 	 SalientVentralAttention
FDR-corrected: 0 	 Coincide with NIBS: 0 	 Limbic
FDR-corrected: 2 	 Coincide with NIBS: 1 	 Control
FDR-corrected: 16 	 Coincide with NIBS: 3 	 Default

Overall:	 FDR-corrected: 58 	 Coincide with NIBS: 18
